In [1]:
import torch
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import japanize_matplotlib
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from dgl.nn import GraphConv,SAGEConv
import dgl
from dgl.data import DGLDataset
from dgl.dataloading import GraphDataLoader
import seaborn as sns
import random
from tqdm import tqdm
import sklearn
from sklearn.model_selection import train_test_split
import os
import yaml
import time
import datetime
from modules import ICPKGIDataset
from models import PatchGCN

In [3]:
'''
データセットを読み込む
データローダーをシャッフルで作成
モデルにデータを入力
1epoch終了後各物体の視線方向ごとのデータを入力し中間層の出力を得る
出力の特徴マップを同じ位置のノードごとの平均をとってその方向の特徴グラフとする
テストデータを同様の手順で処理し特徴グラフを取得する。ただしそれぞれのデータで行う
学習における分類精度と特徴グラフによる分類精度を計算し保存する
分類学習における予測と正解を用いて損失を計算しネットワークを更新する
すべてのテストデータに対して行ったのち全epochについて行う
このファイルは1種類の物体について分類を行うファイルとする
'''

'\nデータセットを読み込む\nデータローダーをシャッフルで作成\nモデルにデータを入力\n1epoch終了後各物体の視線方向ごとのデータを入力し中間層の出力を得る\n出力の特徴マップを同じ位置のノードごとの平均をとってその方向の特徴グラフとする\nテストデータを同様の手順で処理し特徴グラフを取得する。ただしそれぞれのデータで行う\n学習における分類精度と特徴グラフによる分類精度を計算し保存する\n分類学習における予測と正解を用いて損失を計算しネットワークを更新する\nすべてのテストデータに対して行ったのち全epochについて行う\nこのファイルは1種類の物体について分類を行うファイルとする\n'

In [2]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
object_name = 'airplane'  #car bus airplane
setting_file = "config2.yaml"

#データ読み込み
dataset=ICPKGIDataset(f'../data/ICPKGI/8patch_gray_{object_name}.dgl')

#各クラスから均等に10個ずつテスト用として抜き出しtrainデータセットとtestデータセットを作成
labels=[i.item() for _,i in dataset]
traindataset, testdataset, trainlabels, testlabels=train_test_split(dataset,labels,test_size=0.2,shuffle=True,stratify=labels)

#データローダー作成
traindataloader=GraphDataLoader(traindataset,batch_size=16,shuffle=True,num_workers = 0,pin_memory = True)
testdataloader=GraphDataLoader(testdataset,batch_size=10,shuffle=True,num_workers = 0,pin_memory = True)


In [31]:
for i,(g,d) in enumerate(traindataloader):
    break
print(g,d)

Graph(num_nodes=1024, num_edges=6720,
      ndata_schemes={'f': Scheme(shape=(1, 32, 32), dtype=torch.float32)}
      edata_schemes={}) tensor([0, 0, 0, 3, 3, 2, 3, 4, 3, 0, 0, 4, 1, 1, 1, 1])


In [3]:
#設定ファイル読み込み
with open(f'./configs/{setting_file}','r') as f:
    config = yaml.safe_load(f)

#モデルの初期化
model=PatchGCN(1024,[8],5,embedding=True)
model.to(device)

PatchGCN(
  (input_layer): SAGEConv(
    (feat_drop): Dropout(p=0.0, inplace=False)
    (fc_neigh): Linear(in_features=1024, out_features=8, bias=False)
    (fc_self): Linear(in_features=1024, out_features=8, bias=True)
  )
  (middle_layers): ModuleList()
  (output_layer): GraphConv(in=8, out=5, normalization=both, activation=None)
  (m): LeakyReLU(negative_slope=0.01)
  (flatt): Flatten(start_dim=1, end_dim=-1)
)

In [19]:
emb_graphs=[]
emb_labels=[]
for i,(batched_graph,labels) in enumerate(traindataloader):
    emb_labels.extend(labels.tolist())
    batched_graph = batched_graph.to(device)
    labels = labels.to(device)

    pred,emb = model(batched_graph,batched_graph.ndata['f'])
    emb_graphs.extend(dgl.unbatch(emb))
    if i == 1:
        break
print(i)

1


In [13]:
print(emb)

Graph(num_nodes=1024, num_edges=6720,
      ndata_schemes={'f': Scheme(shape=(1, 32, 32), dtype=torch.float32), 'emb': Scheme(shape=(8,), dtype=torch.float32), 'h': Scheme(shape=(5,), dtype=torch.float32)}
      edata_schemes={})


In [14]:
unbatch_emb=dgl.unbatch(emb)
print(unbatch_emb)

[Graph(num_nodes=64, num_edges=420,
      ndata_schemes={'f': Scheme(shape=(1, 32, 32), dtype=torch.float32), 'emb': Scheme(shape=(8,), dtype=torch.float32), 'h': Scheme(shape=(5,), dtype=torch.float32)}
      edata_schemes={}), Graph(num_nodes=64, num_edges=420,
      ndata_schemes={'f': Scheme(shape=(1, 32, 32), dtype=torch.float32), 'emb': Scheme(shape=(8,), dtype=torch.float32), 'h': Scheme(shape=(5,), dtype=torch.float32)}
      edata_schemes={}), Graph(num_nodes=64, num_edges=420,
      ndata_schemes={'f': Scheme(shape=(1, 32, 32), dtype=torch.float32), 'emb': Scheme(shape=(8,), dtype=torch.float32), 'h': Scheme(shape=(5,), dtype=torch.float32)}
      edata_schemes={}), Graph(num_nodes=64, num_edges=420,
      ndata_schemes={'f': Scheme(shape=(1, 32, 32), dtype=torch.float32), 'emb': Scheme(shape=(8,), dtype=torch.float32), 'h': Scheme(shape=(5,), dtype=torch.float32)}
      edata_schemes={}), Graph(num_nodes=64, num_edges=420,
      ndata_schemes={'f': Scheme(shape=(1, 32, 32), 

In [15]:
print(unbatch_emb[0].ndata['emb'].shape)
print(labels)

torch.Size([64, 8])
tensor([3, 1, 0, 4, 3, 3, 2, 2, 2, 1, 4, 4, 0, 3, 4, 4], device='cuda:1')


In [20]:
print(len(emb_graphs))
print(len(emb_labels))
print(emb_labels)

32
32
[4, 0, 3, 3, 3, 2, 0, 3, 3, 1, 4, 3, 4, 3, 3, 3, 1, 3, 2, 3, 3, 3, 3, 3, 0, 2, 3, 3, 4, 4, 1, 0]


In [49]:
direction_graphs=torch.zeros(5,64,8) #中間層の出力のクラス特徴を保存するリスト
direction_labels=[0]*5 #ラベル数が何枚あるかを示すリスト
for i in emb_labels:
    direction_labels[i]+=1
for emb_graph,emb_label in zip(emb_graphs,emb_labels):
    #中間層の出力をラベル数で割ってクラスインデックスに加算する
    direction_graphs[emb_label]+=((emb_graph.ndata['emb'])/direction_labels[emb_label]).to('cpu')


In [51]:
print(direction_graphs.shape)
print(direction_labels)

torch.Size([5, 64, 8])
[4, 3, 3, 17, 5]


In [103]:

i1=torch.randn(5,4,2)
i2=torch.randn(2,1,4,2)
cos=nn.CosineSimilarity(-1)
out=cos(i1,i2)
print(out)
print(torch.sum(out,2))

tensor([[[-0.6558, -0.7974, -0.0419,  0.1725],
         [ 0.5432, -0.0453, -0.8530,  0.2272],
         [-0.8122,  0.0120,  0.9431,  0.6192],
         [ 0.2029, -0.6699, -0.4704,  0.9742],
         [ 0.9992,  0.9900,  0.9499, -0.9605]],

        [[ 0.1627, -0.9442, -0.0737,  0.6008],
         [-0.0224,  0.8065,  0.9075,  0.6445],
         [ 0.3859,  0.8390, -0.9752,  0.9082],
         [-0.6861,  0.2468,  0.5691,  0.9706],
         [-0.8302,  0.6658, -0.9796, -0.9822]]])
tensor([[-1.3225, -0.1279,  0.7620,  0.0367,  1.9787],
        [-0.2544,  2.3360,  1.1579,  1.1003, -2.1262]])


In [107]:
print(out.reshape(2,5,2,2))

tensor([[[[-0.6558, -0.7974],
          [-0.0419,  0.1725]],

         [[ 0.5432, -0.0453],
          [-0.8530,  0.2272]],

         [[-0.8122,  0.0120],
          [ 0.9431,  0.6192]],

         [[ 0.2029, -0.6699],
          [-0.4704,  0.9742]],

         [[ 0.9992,  0.9900],
          [ 0.9499, -0.9605]]],


        [[[ 0.1627, -0.9442],
          [-0.0737,  0.6008]],

         [[-0.0224,  0.8065],
          [ 0.9075,  0.6445]],

         [[ 0.3859,  0.8390],
          [-0.9752,  0.9082]],

         [[-0.6861,  0.2468],
          [ 0.5691,  0.9706]],

         [[-0.8302,  0.6658],
          [-0.9796, -0.9822]]]])


In [65]:
F.cosine_similarity(i1[0],i2[0],0)

tensor(-0.7885)